In [11]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head()

,station,train_name,final_destination_station,arrival_planned_time,arrival_time_delta_in_min,departure_planned_time,departure_time_delta_in_min,planned_platform,changed_platform,stop_canceled,train_type,train_line_ride_id,train_line_station_num
0,Aachen Hbf,RE 4,Aachen Hbf,2024-05-12 00:37:00,60.0,NaN,0.0,7,NaN,True,RE,-2351366154733574977-2405112209,24
1,Aachen Hbf,RE 1,Hamm(Westf)Hbf,NaN,0.0,2024-05-12 00:51:00,2.0,2,NaN,False,RE,1648857726819354655-2405120051,1
2,Aachen Hbf,RE 18,Aachen Hbf,2024-05-12 00:45:00,5.0,NaN,0.0,8,NaN,False,RE,-5376151638251295800-2405112354,9
3,Aachen Hbf,RB 33,Aachen Hbf,2024-05-12 00:27:00,0.0,NaN,0.0,6,NaN,False,RB,3877311608492123870-2405112216,31
4,Aachen Hbf,RE 18,Heerlen,NaN,0.0,2024-05-12 00:17:00,0.0,6,NaN,False,RE,349781417030375472-2405120017,1


In [12]:
df["train_type"].unique()
#df[df["train_line_ride_id"] == "-4092447525890775957-2405101632"]

array(['RE', 'RB', 'S', 'Bus', 'ICE', 'EST', 'HLB', 'IC', 'BRB', 'D',
       'NJ', 'EN', 'TGV', 'EC', 'NBE', 'ag', 'NWB', 'WFB', 'FLX', 'MEX',
       'VIA', 'ENO', 'erx', 'ME', 'SBB', 'IRE', 'SVG', 'RRB', 'RJX',
       'DBK', 'STN', 'ECE', 'SWE', 'UEX', 'N', 'SAB', 'ALX', 'AKN', 'WB',
       'RJ', 'RT', 'TL', 'TLX', 'R', 'CB', 'HBX', 'FEX', 'OPB', 'EVB',
       'ES'], dtype=object)

In [13]:
df[df["stop_canceled"]]

,station,train_name,final_destination_station,arrival_planned_time,arrival_time_delta_in_min,departure_planned_time,departure_time_delta_in_min,planned_platform,changed_platform,stop_canceled,train_type,train_line_ride_id,train_line_station_num
0,Aachen Hbf,RE 4,Aachen Hbf,2024-05-12 00:37:00,60.0,NaN,0.0,7,NaN,True,RE,-2351366154733574977-2405112209,24
31,Aachen Hbf,RE 4,Dortmund Hbf,NaN,0.0,2024-05-12 07:22:00,60.0,7,NaN,True,RE,8841551160963265239-2405120722,1
129,Aschaffenburg Hbf,ICE 1022,Dortmund Hbf,2024-05-12 07:04:00,180.0,2024-05-12 07:05:00,180.0,8,NaN,True,ICE,4250821308365947570-2405120453,3
441,Bamberg,RE 19,Coburg,2024-05-12 06:52:00,60.0,2024-05-12 06:54:00,60.0,4,NaN,True,RE,-1396555714718086421-2405120610,6
453,Bamberg,RE 19,Bamberg,2024-05-12 07:54:00,60.0,NaN,0.0,8,NaN,True,RE,4492277971898031492-2405120730,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109907,Potsdam Hbf,RE 1,Frankfurt(Oder),2024-05-11 02:09:00,60.0,2024-05-11 02:11:00,60.0,4,NaN,True,RE,-4887480265303675289-2405110042,18
109908,Potsdam Hbf,RE 1,Brandenburg Hbf,2024-05-11 02:06:00,60.0,2024-05-11 02:08:00,60.0,3,NaN,True,RE,959474257007241311-2405110024,19
109909,Potsdam Hbf,RE 1,Frankfurt(Oder),2024-05-11 03:43:00,60.0,2024-05-11 03:45:00,60.0,4,4,True,RE,-8084248573434942668-2405110315,7
109913,Potsdam Hbf,RE 1,Frankfurt(Oder),2024-05-11 04:47:00,60.0,2024-05-11 04:50:00,60.0,4,NaN,True,RE,-9094095114882114057-2405110417,7


In [21]:
df_local = df[df["train_type"].isin(["RE", "RB"])]
train_in_dortmund = df_local[df_local["station"] == "Dortmund Hbf"]
train_in_duesseldorf = df_local[df_local["station"] == "Düsseldorf Hbf"]

# Merge/join the two DataFrames on 'train_line_id' to compare 'train_line_station_num'
merged_df = train_in_duesseldorf.merge(
    train_in_dortmund[['train_line_ride_id', 'train_line_station_num']],
    on='train_line_ride_id',
    suffixes=('_duesseldorf', '_dortmund')
)

# Filter the merged DataFrame based on the comparison of 'train_line_station_num'
filtered_df = merged_df[merged_df['train_line_station_num_duesseldorf'] > merged_df['train_line_station_num_dortmund']]

mean_arrival_time_delta = filtered_df.groupby("train_name")["arrival_time_delta_in_min"].median()
mean_arrival_time_delta.reset_index()

,train_name,arrival_time_delta_in_min
0,RE 1,9.5
1,RE 3,0.0
2,RE 4,2.0
3,RE 6,17.0


In [9]:
import pandas as pd

# Example dictionary
TRAIN_TYPE_TO_COMPENSATED_DELAY_FOR_CANCEL = {
    "RE": 60,
    "RB": 40,
    "Bus": 30,
    "S": 30,
    "ICE": 180,
    "IC": 120,
    "FLX": 180,
    "TGV": 180,
}

# Sample data creation for demonstration purposes
# Replace this with your actual data
data = {
    'station': ['Station A', 'Station B', 'Station C'],
    'train_name': ['RE123', 'ICE456', 'Bus789'],
    'final_destination_station': ['Dest A', 'Dest B', 'Dest C'],
    'arrival_planned_time': [pd.Timestamp('2023-01-01 10:00'), pd.Timestamp('2023-01-01 12:00'), pd.NaT],
    'arrival_time_delta_in_min': [0, 0, 0],
    'departure_planned_time': [pd.Timestamp('2023-01-01 10:15'), pd.NaT, pd.NaT],
    'departure_time_delta_in_min': [0, 0, 0],
    'planned_platform': [1, 2, 3],
    'changed_platform': [1, 2, 3],
    'stop_canceled': [True, True, False],
    'train_type': ['RE', 'ICE', 'Bus'],
    'train_line_ride_id': [101, 102, 103],
    'train_line_station_num': [1, 2, 3],
}

df = pd.DataFrame(data)

# Function to get the compensated delay based on the train type
def get_compensated_delay(train_type):
    return TRAIN_TYPE_TO_COMPENSATED_DELAY_FOR_CANCEL.get(train_type, 60)

df

,station,train_name,final_destination_station,arrival_planned_time,arrival_time_delta_in_min,departure_planned_time,departure_time_delta_in_min,planned_platform,changed_platform,stop_canceled,train_type,train_line_ride_id,train_line_station_num
0,Station A,RE123,Dest A,2023-01-01 10:00:00,0,2023-01-01 10:15:00,0,1,1,True,RE,101,1
1,Station B,ICE456,Dest B,2023-01-01 12:00:00,0,NaT,0,2,2,True,ICE,102,2
2,Station C,Bus789,Dest C,NaT,0,NaT,0,3,3,False,Bus,103,3


In [10]:
# Apply the compensated delay to arrival and departure times if the stop is canceled
df.loc[(df['stop_canceled']) & (~df['arrival_planned_time'].isna()), 'arrival_time_delta_in_min'] = df['train_type'].apply(get_compensated_delay)
df.loc[(df['stop_canceled']) & (~df['departure_planned_time'].isna()), 'departure_time_delta_in_min'] = df['train_type'].apply(get_compensated_delay)

df


,station,train_name,final_destination_station,arrival_planned_time,arrival_time_delta_in_min,departure_planned_time,departure_time_delta_in_min,planned_platform,changed_platform,stop_canceled,train_type,train_line_ride_id,train_line_station_num
0,Station A,RE123,Dest A,2023-01-01 10:00:00,60,2023-01-01 10:15:00,60,1,1,True,RE,101,1
1,Station B,ICE456,Dest B,2023-01-01 12:00:00,180,NaT,0,2,2,True,ICE,102,2
2,Station C,Bus789,Dest C,NaT,0,NaT,0,3,3,False,Bus,103,3
